In [1]:
import cv2
import numpy as np
from imutils.video import VideoStream
import imutils
import time

Caffe .prototxt and weights downloaded from https://github.com/LZQthePlane/Face-detection-base-on-ResnetSSD

Pyimagesearch Notebooks used:

[Face Detection](https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/)

[Streaming Video Into OpenCV](https://www.pyimagesearch.com/2019/04/15/live-video-streaming-over-network-with-opencv-and-imagezmq/)

[blob explaination](https://www.pyimagesearch.com/2017/11/06/deep-learning-opencvs-blobfromimage-works/)

[OBS](https://obsproject.com/download) + [OBS virtual cam](https://obsproject.com/forum/resources/obs-virtualcam.949/)

In [2]:
#load serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe("Resnet_SSD_deploy.prototxt", "Res10_300x300_SSD_iter_140000.caffemodel")

# load input image and construct blob resized to 300x300
image = cv2.imread('profpicalex.jpg')
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300) ), 1.0, (300, 300), (104.0, 177.0, 123.0))

# pass blob through network, obtain detects and predictions
print("[INFO] computing object detections...")
net.setInput(blob)
detections = net.forward()

[INFO] loading model...
[INFO] computing object detections...


In [3]:
# loop over detects
for face in range(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the prediction
    confidence = detections[0, 0, face, 2]
    
    #filter out weak detections
    if confidence > 0.5:
        #compute coordinates of bouding box
        box = detections[0,0, face , 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        
        # draw bouding box with probability label
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,0,255), 2)

#show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
#load our model
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe('Resnet_SSD_deploy.prototxt', 'Res10_300x300_SSD_iter_140000.caffemodel')

#initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=1).start()
time.sleep(2.0)

# loop over frames from stream
while True:
    #grab frame and resize to max width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    # grab the frame dim and convert blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    # pass blob through net and obtain detects and pred
    net.setInput(blob)
    detections = net.forward()
    
    #loop through detects
    for face in range(0, detections.shape[2]):
        # extract confidence associated with pred
        confidence = detections[0, 0, face, 2]
        
        #filter ot weak detectons
        if confidence < 0.5:
            continue
            
        # compute coordinates of bounding box
        box = detections[0, 0, face, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        
        # draw bonding box + probability label
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0,0,255), 2)
        cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,0,255), 2)
        
    # show output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    # if the 'q' key was pressed, break from loo
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] loading model...
[INFO] starting video stream...


In [5]:
import time